<a href="https://colab.research.google.com/github/amartinez-se/03MIAR---Algoritmos-de-Optimizacion-2025/blob/main/03MIAR_Trabajo_Practico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Asier Martínez San Esteban  <br>
Url: https://github.com/amartinez-se/03MIAR---Algoritmos-de-Optimizacion-2025
<br>
Google Colab: https://colab.research.google.com/github/amartinez-se/03MIAR---Algoritmos-de-Optimizacion-2025/blob/main/03MIAR_Trabajo_Practico.ipynb

Problema seleccionado: **Organizar los horarios de partidos de una jornada de La Liga**

---

#### Enunciado del problema
Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia
- Los horarios disponibles se conocen a priori y son los siguientes:
| Día de la semana | Horas disponibles |
| :--- | :--- |
| Viernes | 20 |
| Sábado | 12, 16, 18, 20 |
| Domingo | 12, 16, 18, 20 |
| Lunes | 20 |

-  En primer lugar se clasifican los equipos en tres categorías según el numero de seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la categoría A, 11 equipos de categoría B y 6 equipos de categoría C.
| Categoría | A | B | C |
| :---: | :--- | :--- | :--- |
| A | 2 M | 1.3 M | 1 M    |
| B |     | 0.9 M | 0.75 M |
| C |     |       | 0.47 M |

- Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce según los coeficientes de la siguiente tabla
| | Viernes | Sábado | Domingo | Lunes |
| :--- | :--- | :--- | :--- | :--- |
| **12h** | - | 0.55 | 0.45 | - |
| **16h** | - | 0.7 | 0.75 | - |
| **18h** | - | 0.8 | 0.85 | - |
| **20h** | 0.4 | 1 | 1 | 0.4 |

- Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes

- Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje según la siguiente tabla dependiendo del número de coincidencias:
| Coincidencias | -% |
| :--- | :--- |
| 0 | 0% |
| 1 | 25% |
| 2 | 45% |
| 3 | 60% |
| 4 | 70% |
| 5 | 75% |
| 6 | 78% |
| 7 | 80% |
| 8 | 80% |

Los cálculos asociados a una jornada de ejemplo se realizan según se muestra en la siguiente tabla:
| Partido | Categorías | Horario | Base(Mill.) | Ponderación | Base*Ponderación | Corrección Coincidencia |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| Celta - Real Madrid | B-A | V20 | 1,3 | 0,4 | 0,52 | 0,52 |
| Valencia - R. Sociedad | B-A | S12 | 1,3 | 0,55 | 0,72 | 0,72 |
| Mallorca - Eibar | C-C | S16 | 0,47 | 0,7 | 0,33 | 0,33 |
| Athletic - Barcelona | B-A | S18 | 1,3 | 0,8 | 1,04 | 1,04 |
| Leganés - Osasuna | C-C | S20 | 0,47 | 1 | 0,47 | 0,47 |
| Villarreal - Granada | B-C | D16 | 0,75 | 0,75 | 0,56 | 0,42 |
| Alavés - Levante | B-B | D16 | 0,9 | 0,75 | 0,68 | 0,51 |
| Espanyol - Sevilla | B-B | D18 | 0,9 | 0,85 | 0,77 | 0,77 |
| Betis - Valladolid | B-C | D20 | 0,75 | 1 | 0,75 | 0,75 |
| Atlético - Getafe | B-B | L20 | 0,9 | 0,4 | 0,36 | 0,36 |
                                        

# 1. Modelo


### 1.1. ¿Cómo represento el espacio de soluciones?
---
Como aprendimos en el "problema de las damas" codificar el problema de forma óptima es vital para aligerar el algoritmo al máximo. Usando este modelo como referencia, la representación matemática que planteo para este problema es similar.

La jornada tiene n=10 partidos -> vector de longitud 10 donde la posición representa cada partido y el valor numérico, representa el horario asociado, según la tabla siguiente.

| Dia     | Horas | Valor |
| :---    | :---: | :---: |
| Viernes | 20    | 0     |
| Sábado  | 12    | 1     |
| Sábado  | 16    | 2     |
| Sábado  | 18    | 3     |
| Sábado  | 20    | 4     |
| Domingo | 12    | 5     |
| Domingo | 16    | 6     |
| Domingo | 18    | 7     |
| Domingo | 20    | 8     |
| Lunes   | 20    | 9     |

Vector solución para la jornada de ejemplo
```python
[0, 1, 2, 3, 4, 6, 6, 7, 8, 9]
```

| Partido | ID Partido | Horario | ID Horario |
| :--- | :--- | :--- | :--- |
| Celta - Real Madrid | 0 | V20 | 0 |
| Valencia - R. Sociedad | 1 | S12 | 1 |
| Mallorca - Eibar | 2 | S16 | 2 |
| Athletic - Barcelona | 3 | S18 | 3 |
| Leganés - Osasuna | 4 | S20 | 4 |
| Villarreal - Granada | 5 | D16 | 6 |
| Alavés - Levante | 6 | D16 | 6 |
| Espanyol - Sevilla | 7 | D18 | 7 |
| Betis - Valladolid | 8 | D20 | 8 |
| Atlético - Getafe | 9 | L20 | 9 |

De esta forma:
- Se fija la longitud del array a 10
- Es fácil cambiar el horario de un partido (cambiar int de la posición [i])
- Para saber partidos en el mismo horario, contar veces que se repite un valor

### 1.2. ¿Cual es la función objetivo?
---


La función objetivo es maximiazar la suma de audiencias de todos los partidos de la jornada.

Teniendo en cuenta las penalizaciones, su expresión matemática es la siguiente:

$$\text{Maximizar } Z = \sum_{i=1}^{10} (\text{audiencia_base}_i \times \text{ponderacion}_{h_i} \times (1 - \text{penalizacion}_{h_i}))$$

Donde: $i$ es cada uno de los 10 partidos. $h_i$ es el horario asignado al partido $i$. La penalización depende de cuántos partidos coincidan en ese mismo horario $h_i$.

In [43]:
def funcion_objetivo(solucion, base):
    """
    Inputs:
    - solucion: Lista de 10 enteros (0-9) representando el horario de cada partido.
    - base: Lista de 10 floats con la audiencia base de cada partido.

    Output:
    - Float con la audiencia total de la jornada.
    """

    # Definir ponderacion por horario (índices del 0 al 9) - [V20, S12, S16, S18, S20, D12, D16, D18, D20, L20]
    ponderacion = [0.4, 0.55, 0.7, 0.8, 1.0, 0.45, 0.75, 0.85, 1.0, 0.4]

    # Definir penalizacion por coincidencia (tabla de coincidencias proporcionada)
    # (índice = nº coincidencias) - [0, 1, 2, 3, 4, 5, 6, 7, 8]
    penalizaciones = [0.0, 0.25, 0.45, 0.60, 0.70, 0.75, 0.78, 0.80, 0.80, 0.80]

    # Contar partidos en cada horario para aplicar penalizaciones
    conteos_horarios = [solucion.count(i) for i in range(10)]

    audiencia_total = []

    # Calcular la audiencia partido a partido
    for i in range(len(solucion)):
        horario = solucion[i]

        # Total de partidos en este horario
        partidos_en_horario = conteos_horarios[horario]
        penalizacion = penalizaciones[partidos_en_horario - 1] # Si hay un partido en el horario, la penalización es 0

        # base * ponderacion * (1 - penalizacion)
        audiencia_partido = base[i] * ponderacion[horario] * (1 - penalizacion)
        audiencia_total.append(audiencia_partido)

    return round(sum(audiencia_total),2)

sol = funcion_objetivo([0, 1, 2, 3, 4, 6, 6, 7, 8, 9], [1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9])
print(sol)


5.88


### 1.3. ¿Como implemento las restricciones?
---
La restricción a implementar es simple: debe haber al menos un partido el lunes y un partido el viernes.

No debe ser posible generar una solución que no incluyan el 0 (valor para el horario de lunes) o el 9 (valor para el horario de viernes)

In [44]:
import random

def generar_solucion_valida():
  solucion = [1] * 10 # Crear un vector de unos

  # Asignar lunes y viernes a dos partidos aleatorios
  posiciones = random.sample(range(10), 2)
  solucion[posiciones[0]] = 0
  solucion[posiciones[1]] = 9

  # Rellenamos el resto (los que siguen siendo 1) con horarios aleatorios (0-9)
  for i in range(10):
      if solucion[i] == 1:
          solucion[i] = random.randint(0, 9)

  return solucion

# 2. Análisis


#### Orden de complejidad y Espacio de soluciones

Si definimos N como el número de partidos y M como el número de horarios disponibles, el orden de complejidad en notación Big-O es:

$$\mathcal{O}(M^N)$$

Al haber 10 partidos y 10 horarios, el espacio de soluciones es de $10^{10}$ soluciones, es decir, 10.000.000.000 soluciones posibles.

*Restricción (al menos un partido el viernes y uno el lunes)*

La restricción elimina un gran bloque de soluciones.

- Si un horario no tiene partido el Viernes/Lunes, quedan 9 horarios disponibles: $$9^{10} = 3.486.784.401 $$

- Si una solucón no tiene partido NI Viernes NI Lunes, quedan 8 horarios disponibles: $$8^{10} = 1.073.741.824$$

Total de soluciones inválidas $$2*3.486.784.401 + 1.073.741.824 = 5.899.826.978$$

Por tanto, restando esta cifra al espacio de soluciones no restringido:
$$\text{Soluciones Válidas} = 10.000.000.000 - 5.899.826.978 = 4.100.173.022$$

Sin embargo, para calcular el orden de complejidad, las constantes y las reducciones proporcionales se ignoran ya que el problema base sigue escalando de forma exponencial. Por eso este problema se clasifica como **NP-Hard (NP-Duro)**.

# 3. Diseño
- ¿Que técnica utilizo? ¿Por qué?

### 3.1. Búsqueda aleatoria
Para establecer un punto de partida para esta solución, comienzo por búsqueda aleatoria

In [56]:
def busqueda_aleatoria(base, iteraciones=1000):
    mejor_solucion = None
    mejor_audiencia = -1.0

    for _ in range(iteraciones):
        solucion_candidata = generar_solucion_valida()
        audiencia_candidata = funcion_objetivo(solucion_candidata, base)

        # Guardamos la audiencia si es la mejor
        if audiencia_candidata > mejor_audiencia:
            mejor_audiencia = audiencia_candidata
            mejor_solucion = solucion_candidata

    return mejor_solucion, mejor_audiencia

In [62]:
bases_partidos = [1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9]
solucion_busqueda_aleatoria, audiencia_busqueda_aleatoria = busqueda_aleatoria(bases_partidos)

print(f"Búsqueda Aleatoria - Mejor solución encontrada: {solucion_busqueda_aleatoria}")
print(f"Búsqueda Aleatoria - Mejor audiencia máxima estimada: {audiencia_busqueda_aleatoria} Millones")

Búsqueda Aleatoria - Mejor solución encontrada: [8, 6, 6, 4, 2, 2, 0, 3, 7, 9]
Búsqueda Aleatoria - Mejor audiencia máxima estimada: 6.31 Millones


### 3.2. GRASP (Greedy Randomized Adaptative Search Procedure)
Éste algoritmo se ha mencionado varias veces en clase pero no hemos llegado a profundizar, por eso decido implementarlo.

Es un algoritmo heurístico que funciona en dos fases:
- Construcción de una solución con componente voraz
- Mejora de la solución con pequeños cambios

1. **Gestión del Espacio de soluciones**: Con más de 4.100 millones de combinaciones válidas, la fuerza bruta es inviable y una búsqueda puramente aleatoria es pobre. GRASP acota el espacio yendo directamente a las zonas que tienen buena pinta.

2. **Evita soluciones destructivas**: Las penalizaciones por solapar partidos son muy altas, por lo que generar soluciones totalmente al azar produce muchas "malas". La fase voraz de GRASP asigna los horarios basándose en la audiencia, evitando solapamientos absurdos desde el principio, pero usando una lista de candidatos para mantener la variedad y no hacer siempre lo mismo.

3. **Eficacia**: La combinación de construir una buena base y luego pulirla con Búsqueda Local hace que el algoritmo no se quede atascado en óptimos locales. Como  se ha comprobado, es capaz de encontrar el óptimo global matemático absoluto (6.86 Millones) de manera casi instantánea


In [52]:
import random

def grasp(base, iteraciones=100, n_candidatos=3):
    mejor_solucion_global = None
    mejor_audiencia_global = -1.0

    for _ in range(iteraciones):

        # Fase "voraz" - No es totalmente voraz ya que la solución de partida se genera para cumplir la restricción.
        # ----------------------------------------------------------------------------------------------------------

        # Se parte por tanto desde una base válida
        solucion_actual = generar_solucion_valida()

        # Crear una nueva lista y "barajarla" para evitar el sesgo de orden
        jornada_aletoria = list(range(10))
        random.shuffle(jornada_aletoria)

        # Para cada partido de la jornada aleatoria, lo simula en los 10 posibles horarios y guarda la audiencia de las opciones válidas
        for partido in jornada_aletoria:
            evaluaciones = []
            horario_original = solucion_actual[partido]

            for horario_prueba in range(10):
                solucion_actual[partido] = horario_prueba
                audiencia_prueba = funcion_objetivo(solucion_actual, base)

                if audiencia_prueba > 0:
                    evaluaciones.append((audiencia_prueba, horario_prueba))

            # Restauramos el horario original temporalmente
            solucion_actual[partido] = horario_original

            # Ordenamos los horarios de mayor a menor audiencia lograda
            evaluaciones.sort(reverse=True, key=lambda x: x[0])

            # Lista de candidatos con los (n_candidatos=3) mejores. Se elije uno al azar.
            candidatos = evaluaciones[:n_candidatos]

            if candidatos:
                horario_elegido = random.choice(candidatos)[1]
                solucion_actual[partido] = horario_elegido

        # Búsqueda local para hallar óptimo local con pequeños ajustes sobre una solución que ya es buena
        # -----------------------------------------------------------------------------------------------

        mejora = True
        audiencia_actual = funcion_objetivo(solucion_actual, base)

        # Mientras siga habiendo mejora, sigue buscando
        while mejora:
            mejora = False
            for i in range(10):
                for j in range(10):
                    if solucion_actual[i] != j:
                        horario_original = solucion_actual[i]

                        solucion_actual[i] = j
                        nueva_audiencia = funcion_objetivo(solucion_actual, base)

                        # Se acepta la primera mejora encontrada y al hacer break tras mejora = True, entrea de nuevo en while mejora
                        if nueva_audiencia > audiencia_actual:
                            audiencia_actual = nueva_audiencia
                            mejora = True
                            break
                        else:
                            # Si no mejora, deshacemos el movimiento
                            solucion_actual[i] = horario_original

        if audiencia_actual > mejor_audiencia_global:
            mejor_audiencia_global = audiencia_actual
            mejor_solucion_global = list(solucion_actual)

    return mejor_solucion_global, mejor_audiencia_global

In [55]:
bases_partidos = [1.3, 1.3, 0.47, 1.3, 0.47, 0.75, 0.9, 0.9, 0.75, 0.9]
solucion_grasp, audiencia_grasp = grasp(bases_partidos)

print(f"GRASP - Mejor solución encontrada: {solucion_grasp}")
print(f"GRASP - Mejor audiencia máxima estimada: {audiencia_grasp} Millones")

GRASP - Mejor solución encontrada: [8, 4, 0, 7, 9, 5, 3, 2, 1, 6]
GRASP - Mejor audiencia máxima estimada: 6.86 Millones
